In [55]:
import pandas as pd
import pickle
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import time
from mapie.regression import MapieRegressor
import xgboost as xgb
from xgboost import XGBRegressor


# To calculate coverage score
from mapie.metrics import regression_coverage_score

In [56]:
traffic = pd.read_csv("Traffic_Volume.csv")
traffic.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,date_time,traffic_volume
0,NaN,288.28,0.0,0.0,40,Clouds,10/2/12 9:00,5545
1,NaN,289.36,0.0,0.0,75,Clouds,10/2/12 10:00,4516
2,NaN,289.58,0.0,0.0,90,Clouds,10/2/12 11:00,4767
3,NaN,290.13,0.0,0.0,90,Clouds,10/2/12 12:00,5026
4,NaN,291.14,0.0,0.0,75,Clouds,10/2/12 13:00,4918


In [57]:
traffic.dropna(inplace = True)
traffic.info()


<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 126 to 47331
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   holiday         61 non-null     object 
 1   temp            61 non-null     float64
 2   rain_1h         61 non-null     float64
 3   snow_1h         61 non-null     float64
 4   clouds_all      61 non-null     int64  
 5   weather_main    61 non-null     object 
 6   date_time       61 non-null     object 
 7   traffic_volume  61 non-null     int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 4.3+ KB


In [58]:
traffic['date_time'] = pd.to_datetime(traffic['date_time'])
traffic['month'] = traffic['date_time'].dt.month
traffic['weekday'] = traffic['date_time'].dt.weekday
traffic['hour'] = traffic['date_time'].dt.hour

# Drop the original 'date_time' column if no longer needed
traffic.drop(columns=['date_time'], inplace=True)


In [59]:
target = traffic['traffic_volume']
predictors = traffic.drop(columns = ['traffic_volume'])
features_encode = pd.get_dummies(predictors, columns = ['holiday','weather_main'])
features_encode.head()
training_columns = features_encode.columns.tolist()
with open("training_columns.pkl", "wb") as f:
    pickle.dump(training_columns, f)

In [60]:
train_X, test_X, train_y, test_y = train_test_split(features_encode,target,test_size=.3,random_state = 1)
reg = XGBRegressor(random_state = 1)
start = time.time()            # Start Time
reg.fit(train_X, train_y)  
stop = time.time()             # End Time
print(f"Training time: {stop - start}s")
y_pred = reg.predict(test_X)


Training time: 0.11601376533508301s


In [61]:
mapie = MapieRegressor(estimator = reg, # Prediction model to use
                       n_jobs = -1,
                       random_state = 42)

# Fit mapie regressor on training data
start = time.time()  
mapie.fit(train_X, train_y)
stop = time.time()             
print(f"Training time: {stop - start}s")

alpha = 0.1 # For 90% confidence level

# Use mapie.predict() to get predicted values and intervals
y_test_pred, y_test_pis = mapie.predict(test_X, alpha = alpha)

Training time: 8.062741041183472s


In [62]:
predictions = test_y.to_frame()
predictions.columns = ['Actual Value']
predictions["Predicted Value"] = y_test_pred.round(2)
predictions["Lower Value"] = y_test_pis[:, 0].round(2)
predictions["Upper Value"] = y_test_pis[:, 1].round(2)

# Take a quick look
predictions.tail(5)

,Actual Value,Predicted Value,Lower Value,Upper Value
20345,1513,1421.810059,354.03,1622.44
47330,962,969.010010,640.87,1432.16
18946,494,1019.229980,490.35,1407.46
9455,615,557.390015,171.91,1087.70
40656,600,750.539978,300.52,1106.83


In [63]:
coverage = regression_coverage_score(test_y,           # Actual values
                                     y_test_pis[:, 0], # Lower bound of prediction intervals
                                     y_test_pis[:, 1]) # Upper bound of prediction intervals

coverage_percentage = coverage * 100
print(f"Coverage: {coverage_percentage:.2f}%")

Coverage: 94.74%


In [64]:
sorted_predictions = predictions.sort_values(by=['Actual Value']).reset_index(drop=True)

# Create a figure and axis object with specified size and resolution
fig, ax = plt.subplots(figsize=(8, 4))

# Plot the actual values with green dots
plt.plot(sorted_predictions["Actual Value"], 'go', markersize=3, label="Actual Value")

# Fill the area between the lower and upper bounds of the prediction intervals with semi-transparent green color
plt.fill_between(np.arange(len(sorted_predictions)),
                 sorted_predictions["Lower Value"],
                 sorted_predictions["Upper Value"],
                 alpha=0.2, color="green", label="Prediction Interval")

# Set font size for x and y ticks
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Set the limit for the x-axis to cover the range of samples
plt.xlim([0, len(sorted_predictions)])

# Label the x-axis and y-axis with appropriate font size
plt.xlabel("Samples", fontsize=10)
plt.ylabel("Target", fontsize=10)

# Add a title to the plot, including the coverage percentage, with bold formatting
plt.title(f"Prediction Intervals and Coverage: {coverage_percentage:.2f}%", fontsize=12, fontweight="bold")

# Add a legend to the plot, placed in the upper left, with specified font size
plt.legend(loc="upper left", fontsize=10);
plt.savefig('coverage.svg')

In [65]:
importance = reg.feature_importances_

# Storing feature importance as a dataframe
feature_imp = pd.DataFrame(list(zip(train_X.columns, importance)),
               columns = ['Feature', 'Importance'])

feature_imp = feature_imp.sort_values('Importance', ascending = False).reset_index(drop = True)

# Bar plot
plt.figure(figsize=(10, 5))
plt.barh(feature_imp['Feature'], feature_imp['Importance'], color = ['purple', 'pink'])

plt.xlabel("Importance")
plt.ylabel("Input Feature")
plt.title('Which features are the most important for traffic volume prediction?') 
plt.tight_layout()
plt.savefig("feature_imp.svg");

In [66]:
all_residuals = test_y - y_pred

# Set up the figure with custom size and resolution (DPI)
plt.figure(figsize=(6, 4), dpi = 150)

# Plot the histogram of residuals
plt.hist(all_residuals, bins = 25, color = 'lime', edgecolor = 'black')

# Label X and Y axes
plt.xlabel('Residuals', fontsize = 14)
plt.ylabel('# of Test Datapoints', fontsize = 14)

# Set the title of the plot
plt.title('Distribution of Residuals', fontsize = 16)

# Adjust the font size of x and y ticks
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10);
plt.savefig('residual_plot.svg')

In [67]:
plt.figure(figsize = (6, 4), dpi = 150)

# Scatter plot of actual vs predicted values
plt.scatter(test_y, y_pred, color = 'blue', alpha = 0.6, edgecolor = 'black', s = 40)

# 45-degree reference line (perfect predictions)
plt.plot([min(test_y), max(test_y)], [min(test_y), max(test_y)], color = 'red', linestyle = '--', lw = 2)

# Axis labels and title
plt.xlabel('Actual Values', fontsize = 10)
plt.ylabel('Predicted Values', fontsize = 10)
plt.title('Predicted vs Actual Values', fontsize = 12)

# Adjust ticks
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10);
plt.savefig('pred_vs_actual.svg')

In [68]:
mapie_pickle = open('mapie.pickle', 'wb') 

pickle.dump(mapie, mapie_pickle) 

# Close the file
mapie_pickle.close()